![ups logo](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png/640px-Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png)

## Aprendizaje Automático P64

## Practica 04


### Árboles de decisión, Random Forest (RF) y Máquinas de Soporte Vectorial (SVM)

### Fase 1: Analisis exploratorio y preparacionde los datos

### Autor: Diego Tapia